In [114]:
def coding(text):
  import numpy as np
  import zlib
  crc32=zlib.crc32(text.encode('utf-8'))
  a=[]
  for elem in text:
    a.append(ord(elem))
  x=np.zeros((len(a)//3+min(len(a)%3,1),85))
  for index in range(0,len(a),3):
    b=np.zeros(20)
    for i in range(20):
      if 2**(19-i)<=a[index]:
        b[i]=1
        a[index]-=2**(19-i)
    k=0
    for i in range(25):
     if 2**k==i+1:
      k+=1
     else:
      x[index//3][i]=b[i-k]
    b=np.zeros(20)
    if index+1<len(a):
      for i in range(20):
        if 2**(19-i)<=a[index+1]:
          b[i]=1
          a[index+1]-=2**(19-i)
    for i in range(20):
      if i<6:
       x[index//3][i+25]=b[i]
      else:
       x[index//3][i+26]=b[i]      
    b=np.zeros(20)
    if index+2<len(a):
      for i in range(20):
        if 2**(19-i)<=a[index+2]:
          b[i]=1
          a[index+2]-=2**(19-i)      
    for i in range(20):
      if i<17:
       x[index//3][i+46]=b[i]
      else:
       x[index//3][i+47]=b[i]   
  p=[]      
  for ind in range(x.shape[0]):
    y=np.zeros(7)    
    for i in range(7):
      s=0
      for j in range(2**i-1, 85, 2**(i+1)):
        for k in range(2**i):
          if j+k<85:
            s+=x[ind][j+k]
      y[i]=int(s)%2
    for i in range(7):        
      x[ind][2**i-1]=y[i]
    p.append(y)
  cons=zlib.crc32(str(p).encode('utf-8'))
  return x, p, crc32, cons

In [115]:
def encoding(mes, matr, c32, cs):
  
  import numpy as np
  import zlib
   
  message, mat=mes.copy(), matr.copy()
  if cs==zlib.crc32(str(mat).encode('utf-8')):
    print('контрольная сумма корректна') 
  def con_sum(message):
    p=[]    
    for ind in range(message.shape[0]):
      y=np.zeros(7)  
      for i in range(7):
        s=0
        for j in range(2**i-1, 85, 2**(i+1)):
          for k in range(2**i):
            if j+k<85:
              s+=message[ind][j+k]
        y[i]=int(s)%2
      p.append(y)
    return(p)
  def go_back(x):
    k=0
    w1=[]
    w2=[]
    w3=[]
    for i in range(25):
      if 2**k==i+1:
        k+=1
      else:
        w1.append(x[i])
    for i in range(25,46):
      if 2**k==i+1:
        k+=1
      else:
        w2.append(x[i])
    for i in range(46,85):
      if 2**k==i+1:
        k+=1
      else:
        w3.append(x[i])    
    l1=0    
    for i in range(len(w1)):
      l1+=2**(19-i)*w1[i]
    l=chr(int(l1))
    l2=0    
    for i in range(len(w2)):
      l2+=2**(19-i)*w2[i]
    l3=0    
    for i in range(len(w3)):
      l3+=2**(19-i)*w3[i]    
    if l2!=0:  
      l+=chr(int(l2))
    if l3!=0:
      l+=chr(int(l3))
    return l
  def eq(m1,m2):
    flag=True
    for i in range(len(m1)):
      for j in range (len(m1[0])):
        if m1[i][j]!=m2[i][j]:
          flag=False
    return flag
  for ind in range(message.shape[0]):
    for i in range(7):
      message[ind][2**i-1]=0
  p=con_sum(message)
  if (eq(p,mat)):
    text=''
    for ind in range(message.shape[0]):
      text+=go_back(message[ind])
  else: 
    for ind in range(len(p)):
        err=0  
        for i in range(len(p[ind])):
          if p[ind][i]!=mat[ind][i] and err+2**i<=85:
            err+=2**i
        if err>0:
          message[ind][err-1]=1-message[ind][err-1]
    p=con_sum(message)
    text=''
    for ind in range(message.shape[0]):
      text+=go_back(message[ind])
  if c32==zlib.crc32(text.encode('utf-8')):
    print('сообщение доставлено верно')
  else:
    print('в доставленном сообщении имеются ошибки')
  return text

In [116]:
t="""В то время как циклические избыточные коды являются частью стандартов, у этого термина не существует общепринятого определения — трактовки различных авторов нередко противоречат друг другу.

Этот парадокс касается и выбора многочлена-генератора: зачастую стандартизованные полиномы не являются самыми эффективными в плане статистических свойств соответствующего им check redundancy code.

При этом многие широко используемые полиномы не являются наиболее эффективными из всех возможных. В 1993—2004 годах группа учёных занималась исследованием порождающих многочленов разрядности до 16 24 и 32 бит и нашла полиномы, дающие лучшую, нежели стандартизированные многочлены, производительность в смысле кодового расстояния. Один из результатов этого исследования уже нашёл своё применение в протоколе iSCSI.

Самый популярный и рекомендуемый IEEE полином для CRC-32 используется в Ethernet, FDDI; также этот многочлен является генератором кода Хемминга. Использование другого полинома — CRC-32C — позволяет достичь такой же производительности при длине исходного сообщения от 58 бит до 131 кбит, а в некоторых диапазонах длины входного сообщения может быть даже выше, поэтому в наши дни он тоже пользуется популярностью. К примеру, стандарт ITU-T G.hn использует CRC-32C с целью обнаружения ошибок в полезной нагрузке.

Ниже в таблице перечислены наиболее распространённые многочлены — генераторы CRC. На практике вычисление CRC может включать пре- и постинверсию, а также обратный порядок обработки битов. В проприетарных реализациях CRC для усложнения анализа кода применяют ненулевые начальные значения регистров.

Понятие циклические коды - достаточно широкое. В англоязычной литературе CRC понимается двояко в зависимости от контекста: Cyclic Redundancy Code или Cyclic Redundancy Check. Под первым понятием подразумевают математический феномен циклических кодов, под вторым — конкретное применение этого феномена как хеш-функции.

Циклические коды не только просты в реализации, но и обладают тем преимуществом, что подходят для обнаружения пакетных ошибок: непрерывных последовательностей ошибочных символов данных в сообщениях. Это важно, потому что пакетные ошибки являются распространёнными ошибками передачи во многих каналах связи, включая магнитные и оптические запоминающие устройства.


"""

БЕЗ ОШИБОК

In [117]:
mes, matr, c32, cs=coding(t) 

In [118]:
te=encoding(mes, matr, c32, cs)

контрольная сумма корректна
сообщение доставлено верно


In [119]:
print(t==te)

True


In [120]:
def eq(m1,m2):
    flag=True
    for i in range(len(m1)):
      for j in range (len(m1[0])):
        if m1[i][j]!=m2[i][j]:
          flag=False
    return flag 

Не более одной ошибки


In [121]:
m1, mat1, c1, cs1=coding(t)
m2, mat2, c2, cs2=coding(t)

In [122]:
import random

In [123]:
#внесение ошибок
for i in range(m1.shape[0]):
  j=random.randint(0,84)
  m1[i][j]=1-m1[i][j]

In [124]:
eq(m1,m2)

False

In [125]:
encoding(m1,mat1,c1,cs1)==encoding(m2,mat2,c2,cs2)

контрольная сумма корректна
сообщение доставлено верно
контрольная сумма корректна
сообщение доставлено верно


True

In [126]:
t==encoding(m1,mat1,c1,cs1)

контрольная сумма корректна
сообщение доставлено верно


True

Более одной ошибки 

In [127]:
m3, mat3, c3, cs3=coding(t)

In [128]:
n=0
import random
for i in range(m3.shape[0]):
  j=random.randint(10,56)
  n+=1
  m3[i][j]=1-m3[i][j]
  r=random.choice([True, False])
  k=random.choice(list(range(-10,10)))
  if r:
    m3[i][j+k]=1-m3[i][j+k]
    if (j<25 and j+k>=25) or (j>=25 and j<46 and (j+k<25 or j+k>=46)) or (j>=46 and j+k<46):
      n+=1
#ошибки допущены в (символах):      
print(n/(m3.shape[0]*3))
print(n)

0.370757180156658
852


In [129]:
dec=encoding(m3,mat3,c3,cs3)

контрольная сумма корректна
в доставленном сообщении имеются ошибки


In [130]:
dec==t

False

In [131]:
err=0
for i in range(len(dec)):
  if dec[i]!=t[i]:
    err+=1
print(err/len(dec))
print(err)

0.2699172834131476
620


In [132]:
#исправлено ошибок 
print(n-err)

232


In [133]:
tr=0
f=0
for i in range(0,len(t)-2,3):
  if dec[i]==t[i] and dec[i+1]==t[i+1] and dec[i+2]==t[i+2]:
    tr+=1
  else:
    f+=1

In [134]:
print(tr,f)

421 344
